<H2>Estrutura em Python</H2>
<li><lu>Puxar arquivos Views</li>
<li>Tratar duplicados</li>
<li>Tratar Nulos</li>
<li>Formatar Datas</lu></li>

In [8]:
# Bibliotecas
# sqlalchemy
# Pandas
# dotenv
# urlib

In [2]:
!pip install python-dotenv
!pip install psycopg2-binary

In [1]:
from sqlalchemy import create_engine
import urllib.parse
import pandas as pd
from dotenv import dotenv_values
from decimal import Decimal 

In [2]:
# Carregar credenciais
config = dotenv_values(".env")

# Leitura das credenciais
user = config["DB_USER"]
password = urllib.parse.quote_plus(config["DB_PASS"])
host = config["DB_HOST"]
port = config["DB_PORT"]
database = config["DB_NAME"]

In [3]:
# Conexão
conexao = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
# Criando a engine
engine = create_engine(conexao)

In [4]:
# Criando uma função para ler as Views
def carregar_views(querys):
    df = pd.read_sql(f'SELECT * FROM sales.{querys}', engine)
    return df

In [58]:
faixa_etaria_clientes = carregar_views("faixa_etaria_clientes")
faturamento_acumulado = carregar_views("faturamento_acumulado")
faturamento_mensal = carregar_views("faturamento_mensal")
rank_receita_estado_loja = carregar_views("rank_receita_estado_loja")
top5_produtos_vendidos = carregar_views("top5_produtos_vendidos")

<h3>Explorando valores das colunas das Views</h3>

In [6]:
faixa_etaria_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   qtd_clientes  6 non-null      int64 
 1   faixa_etaria  6 non-null      object
 2   percentual    6 non-null      object
dtypes: int64(1), object(2)
memory usage: 276.0+ bytes


In [7]:
faturamento_acumulado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   mes              12 non-null     datetime64[ns, UTC]
 1   venda_mensal     13 non-null     float64            
 2   venda_acumulada  13 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(2)
memory usage: 444.0 bytes


In [8]:
faturamento_mensal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   mes                 12 non-null     float64
 1   faturamento_mensal  12 non-null     object 
dtypes: float64(1), object(1)
memory usage: 324.0+ bytes


In [9]:
rank_receita_estado_loja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   regiao           69 non-null     object
 1   loja_nome        69 non-null     object
 2   receita          69 non-null     object
 3   ranking_regiao   69 non-null     int64 
 4   ranking_receita  69 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.8+ KB


In [10]:
top5_produtos_vendidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   marca        5 non-null      object
 1   quantidade   5 non-null      int64 
 2   rank_vendas  5 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 252.0+ bytes


# Dataframes para serem observados:
> faturamento_acumulado<br>
> faturamento_mensal<br>
> rank_receita_estado_loja

<h5>DF Faturamento Acumulado</h5>
<li >Converter data, e padronizar valores monetários em Decimal</li>

In [16]:
# Converter data, tratar nulo e padronizar valores monetários em Decimal
faturamento_acumulado.head()

,mes,venda_mensal,venda_acumulada
0,2020-09-01 03:00:00+00:00,313050.0,313050.0
1,2020-10-01 03:00:00+00:00,1966170.0,2279220.0
2,2020-11-01 03:00:00+00:00,2709150.0,4988370.0
3,2020-12-01 03:00:00+00:00,2997586.0,7985956.0
4,2021-01-01 03:00:00+00:00,2713320.0,10699276.0


In [59]:
faturamento_acumulado['mes'] = faturamento_acumulado['mes'].dt.date
faturamento_acumulado['mes'] = faturamento_acumulado['mes'].fillna("Null")
faturamento_acumulado['venda_acumulada'] = faturamento_acumulado['venda_acumulada'].apply(lambda x: Decimal(str(x)))
faturamento_acumulado.drop(columns='venda_mensal')

,mes,venda_acumulada
0,2020-09-01,313050.0
1,2020-10-01,2279220.0
2,2020-11-01,4988370.0
3,2020-12-01,7985956.0
4,2021-01-01,10699276.0
5,2021-02-01,14863498.0
6,2021-03-01,24114067.0
7,2021-04-01,32983030.0
8,2021-05-01,58064353.0
9,2021-06-01,96543781.0


<h5>DF Faturamento Mensal</h5>
<li >Converter data para inteiro, e padronizar valores monetários em Decimal</li>

In [101]:
faturamento_mensal['faturamento_mensal'] = (
    faturamento_mensal['faturamento_mensal']
    .str.replace('R$','', regex = False)
    .str.strip()
    .str.replace(',', '.', regex = False)
    .str.replace('.', '', regex = False)
    .astype(float)
)
faturamento_mensal['mes'] = faturamento_mensal['mes'].astype(int)
faturamento_mensal['faturamento_mensal'] = faturamento_mensal['faturamento_mensal'].apply(lambda x: Decimal(str(x)))
faturamento_mensal

,mes,faturamento_mensal
0,1,271332000.0
1,2,416422200.0
2,3,925056900.0
3,4,886896300.0
4,5,2508132300.0
5,6,3847942800.0
6,7,6772747000.0
7,8,7908712900.0
8,9,31305000.0
9,10,196617000.0


<h5>rank_receita_estado_loja</h5>
<li >Padronizar valores monetários em Decimal</li>

In [128]:
rank_receita_estado_loja.head()

,regiao,loja_nome,receita,ranking_regiao,ranking_receita
0,SP,RONDINELIA HENDRI WALTERCY LTDA,"R$ 29.624.679,00",1,1
1,SP,IGNACIA ROSIDELMA PAVLA LTDA,"R$ 25.199.536,00",2,2
2,SP,GERRIVALDO ROSIELEN VALTEIDE LTDA,"R$ 24.649.108,00",3,3
3,SC,CLAUDINEO JOZENAIDE LUYANE LTDA,"R$ 22.276.986,00",1,4
4,RS,KIYOKO CILEIDI JERY LTDA,"R$ 13.108.117,00",1,12


In [135]:
rank_receita_estado_loja['receita'] = (
    rank_receita_estado_loja['receita']
    .str.replace('R$', '', regex = False)
    .str.strip()
    .str.replace(',','.',regex = False)
    .str.replace('.','',regex = False)
    .astype(float)
)
rank_receita_estado_loja['receita'] = rank_receita_estado_loja['receita'].apply(lambda x: Decimal(str(x)))
rank_receita_estado_loja

,regiao,loja_nome,receita,ranking_regiao,ranking_receita
0,SP,RONDINELIA HENDRI WALTERCY LTDA,2962467900.0,1,1
1,SP,IGNACIA ROSIDELMA PAVLA LTDA,2519953600.0,2,2
2,SP,GERRIVALDO ROSIELEN VALTEIDE LTDA,2464910800.0,3,3
3,SC,CLAUDINEO JOZENAIDE LUYANE LTDA,2227698600.0,1,4
4,RS,KIYOKO CILEIDI JERY LTDA,1310811700.0,1,12
...,...,...,...,...,...
64,PI,COSMIA IDERLANDO VALTILENE LTDA,4980000.0,1,3099
65,TO,NORMA JUCEILDO FILEMOM LTDA,4785000.0,2,3124
66,TO,SUSIETE LIVE DUCIVAN LTDA,4498500.0,3,3200
67,PI,ADANEUZA OLGAIDE ADELNA LTDA,3585000.0,2,3318


<h4>Exportar os arquivos para CSV</h4>

In [155]:
rank_receita_estado_loja.to_csv('rank_receita_estado_loja.csv', index=False, sep=';')
faturamento_acumulado.to_csv('faturamento_acumulado.csv', index=False, sep=';')
faturamento_mensal.to_csv('faturamento_mensal.csv', index=False, sep=';')
top5_produtos_vendidos.to_csv('top5_produtos_vendidos.csv', index=False, sep=';')
faixa_etaria_clientes.to_csv('faixa_etaria_clientes.csv', index=False, sep=';')

<h3>ETL finalizado</h3>